# Cafe Sale Scan Rates Analysis

This notebook aims to analyse the Sale scan rates at different cafes post 22nd Jan 2020, comparing total scans each day with number of used cups at each cafe from the manual data in Google Spreadsheet.

Author: Arihant Jain

Date: 12th October 2020

### Fetching Data
#### Import libraries

In [1]:
from requests import get
from pandas import DataFrame, to_datetime, read_csv, merge, melt
from time import time
from plotly.graph_objects import Figure, Scatter
from plotly.express import box, colors
from dotenv import load_dotenv
from datetime import datetime
import json, os
import numpy as np

ModuleNotFoundError: No module named 'plotly'

#### Set how many days old the data can be

In [ ]:
fetchSaleData = True

# for 1 day
lastUpdatedInDays = 60

#### Check if data is present locally and if it can be used

In [ ]:
# current time
currentTime = time()
timeToCheckAgainst = datetime.fromtimestamp(currentTime - lastUpdatedInDays*24*60*60)

if not os.path.exists('../data files'):
    os.makedirs('../data files')

print('Checking for sale data...')
if os.path.exists("../data files/rawSaleData.json"):
    saleDataTime = datetime.fromtimestamp(os.path.getmtime('../data files/rawSaleData.json'))
    print('Found sale data from:', saleDataTime.strftime('%c'))
    
    # if older than 'lastUpdatedInDays' days, fetch again
    if saleDataTime <= timeToCheckAgainst:
        print('Locally saved data is older than {} hours!'.format(lastUpdatedInDays*24))
        print('Have to fetch again...')
        fetchSaleData = True
    else:
        print('Will be using this locally saved data...')
        fetchSaleData = False
else:
    print('No local data found!')
    print('Have to fetch again...')
    fetchSaleData = True

#### Get URL from .env if in case we fetch from API

In [ ]:
load_dotenv()

domain = os.getenv("URL")

#### Get data

In [ ]:
if fetchSaleData:
    print('Getting Sale data... Please wait!')
    startTime = time()

    # get sale data
    response = get('https://' + domain + 'sale')
    saleData = response.json()

    print('Took', round(time() - startTime, 2), 'seconds to get Sale data.')
    
    # save locally for later reuse
    with open('../data files/rawSaleData.json', 'w') as fp:
        json.dump(saleData, fp, indent=4)
else:
    # use locally saved data
    with open('../data files/rawSaleData.json', 'r') as fp:
        saleData = json.load(fp)

### Data Prep
#### Working with Sale Data

In [ ]:
# build dataframe
saleDF = DataFrame.from_dict(saleData, orient='columns')

# dropping columns
saleDF = saleDF.drop(['cup_id', 'id', 'scanned_at', 'return_id', 'scanned_at_melbourne_date_time', 'scanned_at_melbourne_time'], axis=1)

# renaming columns
saleDF = saleDF.rename(columns={"scanned_at_melbourne_date": "date"})

# converting date column to datetime64 datatype
saleDF['date'] = to_datetime(saleDF['date'], dayfirst=True)

# get data post 22nd Jan 2020
saleDF = saleDF[saleDF['date'] >= '2020/01/22']

# get weeks from date
saleDF['week'] = saleDF['date'].dt.isocalendar().week

# drop date
saleDF = saleDF.drop('date', axis=1)

saleDF.head()

#### Grouping by cafes and date

In [ ]:
groupedOnlineSalesDF = saleDF.groupby(['cafe_id','week']).size().reset_index(name='online_count')
groupedOnlineSalesDF.head()

### Get data from Google Spreadsheet

#### Prepare link

In [ ]:
# get spreadsheet link from .env file
spreadsheetLink = os.getenv("Google_Spreadsheet_URL")

# this is the sheet name
sheetName = 'Responses'

# complete url
url = spreadsheetLink + 'gviz/tq?tqx=out:csv&sheet=' + sheetName

#### Get and clean data

In [ ]:
sheetDF = read_csv(url)

# get columns to be deleted programatically
cols = []
for col in sheetDF.columns:
    if 'have' in col.lower() or 'notes' in col.lower() or 'checked' in col.lower() or 'unnamed' in col.lower():
        cols.append(col)

# make list of columns to be deleted
deleteCols = cols + ['Timestamp', 'Name of BorrowCup assistant']
sheetDF = sheetDF.drop(deleteCols, axis=1)
sheetDF = sheetDF.rename(columns={'Date Today': 'Date'})
sheetDF['Date'] = to_datetime(sheetDF['Date'], yearfirst=True)
sheetDF.head()

#### Working with manual return data

In [ ]:
# drop columns
sheetsDF = sheetDF.drop(['Week Number', 'Year', 'Washed'], axis=1)

sheetsDF = sheetsDF[sheetsDF['Date'] >= '2020/01/22']

# rename columns
renamed = {
    'Nesso Café Used': '101',
    'Cafelito Used': '102',
    'Taste Baguette Used': '103',
    'Church of Secular Coffee Used': '104',
    'Swifts Used': '105',
    'Wholefoods Used': '106',
    'Supernatural Eatery Used': '107',
    'Script Used': '108',
    'Cinque Lire Café Used': '109'
}
sheetsDF = sheetsDF.rename(columns=renamed)

sheetsDF.head(10)

In [ ]:
# melt the dataframe so that cafe columns become one column with cafe ids as values
# See https://pandas.pydata.org/docs/reference/api/pandas.melt.html
sheetsDF = melt(sheetsDF, id_vars = sheetsDF.columns[0], value_vars = sheetsDF.columns[1:], var_name='cafe_id', value_name='manual_count')

# get week numbers from Date
sheetsDF['week'] = sheetsDF['Date'].dt.isocalendar().week

# drop date column
sheetsDF = sheetsDF.drop(['Date'], axis=1)

# rearrange columns
sheetsDF = sheetsDF[['cafe_id', 'week', 'manual_count']]

# change cafe_id column type from object to int64
sheetsDF["cafe_id"] = sheetsDF["cafe_id"].astype(str).astype(int)

sheetsDF.head()

In [ ]:
# group by cafe id and week number
groupedManualSalesDF = sheetsDF.groupby(['cafe_id', 'week']).sum().reset_index()

groupedManualSalesDF.head()

In [ ]:
# merging online (left) and manual (right) data with an outer join
mergedDF = merge(left=groupedOnlineSalesDF, right=groupedManualSalesDF, how='outer', on=['cafe_id', 'week'], indicator='present', validate='1:1')

# sort by cafe_id, week
mergedDF = mergedDF.sort_values(by=['cafe_id', 'week'], ignore_index=True)
mergedDF.head()

#### Checking if we have complete missing whole week records in any one data

In [ ]:
mergedDF.present.unique()

`right_only` suggests that, for some cafe-week pairs, online collected data is missing.

#### Let's investigate what are those cafe-week pairs...

In [ ]:
mergedDF[mergedDF['present'] == 'right_only']

So for most cafe-week pairs, the sales have been zero as the manual sales count is zero - for which the online sales count would be obviously non-existent (thus NaN). Furthermore, it can be seen that there are only 2 cafes in 15 of the total 17 intances where manual sales count is zero. These are 102 (Cafelito) and 106 (Wholefoods). This is understandable as these cafes were mostly closed during the small period BorrowCup operated in 2020. Wholefoods was never open and Cafelito may have opened
for a day or two.

#### Groupby converted NaNs to zero on summation

We can see from the distinct values of the manual count below that, upon grouping and summation, for those cafe-week pairs where all manual sales count were only NaNs, those cafe-week pairs were assigned Zero in the output dataframe.

In [ ]:
groupedManualSalesDF.manual_count.unique()

#### Removing such rows where cafe-week pair manual sales count is zero

First check whether all zero manual counts are among the 17 instances seen above.

In [ ]:
mergedDF[mergedDF['manual_count'] == 0]

Yep. It is easier to see that the instances are the same when you look that they are all `right_only` instances. They are 15 in total.

So we can remove them easily.

In [ ]:
mergedDF = mergedDF[mergedDF['manual_count'] != 0]

mergedDF.head()

From the above, we know that there were 17 instances where no online sales was recorded, out of which 15 were removed in the previous step. We now have 2 instances (rows) where there is `right_only` tag.

In [ ]:
mergedDF[mergedDF['present'] == 'right_only']

Above we saw that there are 2 NaNs here.

Let's see if we have more NaNs anywhere else in the dataframe.

In [ ]:
mergedDF[mergedDF.isna().any(axis=1)]

No. We don't have any other NaNs anywhere else. 

Since we know that online count NaNs implies that there was no online data collected for that week. We can set them to Zero for our analysis.

In [ ]:
# replace NaN with zeroes
mergedDF['online_count'] = mergedDF['online_count'].fillna(0)

mergedDF.loc[mergedDF['present'] == 'right_only', :]

#### Get scan percentages (or rates)

Formula:

$$percentage = \frac{online\,count}{manual\,count}\times 100$$

In [ ]:
mergedDF['percent'] = round(mergedDF['online_count']/mergedDF['manual_count']*100)

mergedDF.head()

Now that we have scan rates, let's join this data with cafe names data to read and understand quickly and clearly.

#### Reading cafes data

In [ ]:
if not os.path.exists("../data files/cafes.csv"):
    print('Existing cafes file not found. Run the GetCafesData.ipynb notebook.')
else:
    # read the csv file
    cafesDF = read_csv('../data files/cafes.csv')
    
    # drop columns
    cafesDF = cafesDF.drop(['name', 'latitude', 'longitude'], axis=1)
cafesDF

#### Joining cafes data with sales data

In [ ]:
# inner join
mergedDF = merge(mergedDF, cafesDF, left_on = 'cafe_id', right_on = 'id')

# drop columns
mergedDF = mergedDF.drop(['present', 'id'], axis=1)

# rearrange columns
mergedDF = mergedDF[['cafe_name', 'cafe_id', 'week', 'online_count', 'manual_count', 'percent']]

mergedDF.head()

#### Checking how the data looks

In [ ]:
mergedDF['percent'].describe()

#### Check scan rates spread for each cafe

In [ ]:
fig = box(mergedDF, x="percent", y="cafe_name", color="cafe_name")
fig.update_layout(showlegend=False)

fig.show()

#### Creating line chart for scan rates

In [ ]:
# Create traces
fig = Figure()
for cafe_id in mergedDF['cafe_id'].unique():
    fig.add_trace(Scatter(x=mergedDF['week'], y=mergedDF.loc[mergedDF['cafe_id'] == cafe_id, 'percent'],
                        mode='lines+markers',
                        name=mergedDF.loc[mergedDF['cafe_id'] == cafe_id, 'cafe_name'].unique()[0],
                        line_shape='spline'))

# Edit the layout
fig.update_layout(title='Scan rates per cafe per week line chart',
                   xaxis_title='Week',
                   yaxis_title='Percentage')

fig.show()

In [ ]:
# Create traces
fig = Figure()
cafe_ids = mergedDF['cafe_id'].unique()
colorDict = dict(zip(cafe_ids, colors.qualitative.Plotly))
for cafe_id in cafe_ids:
    
    x = mergedDF.loc[mergedDF['cafe_id'] == cafe_id, 'week']
    y = mergedDF.loc[mergedDF['cafe_id'] == cafe_id, 'percent']
    z = np.polyfit(x, y, 2)
    p = np.poly1d(z)
    mergedDF.loc[mergedDF['cafe_id'] == cafe_id,'regressionPercentage'] = p(x)
    
    fig.add_trace(Scatter(x=x, y=y,
                          mode='markers',
                          name=mergedDF.loc[mergedDF['cafe_id'] == cafe_id, 'cafe_name'].unique()[0],
                          marker_color=colorDict[cafe_id],
                          showlegend=False,
                          marker={'size':8},
                          legendgroup=str(cafe_id)
                         ))

    fig.add_trace(Scatter(x=x, y=p(x),
                          mode='lines',
                          name=mergedDF.loc[mergedDF['cafe_id'] == cafe_id, 'cafe_name'].unique()[0],
                          line_color=colorDict[cafe_id],
                          legendgroup=str(cafe_id)
                         ))

# Edit the layout
fig.update_layout(title='Scan rates per cafe per week line chart',
                  xaxis_title='Week',
                  yaxis_title='Percentage',
                  legend_title_text='Cafes')

fig.show()

In [ ]:
mergedDF.head()

#### Saving scan rates data locally

In [ ]:
mergedDF.to_csv('../data files/saleScanRates.csv', index = False)